In [1]:
import numpy as np
import datetime as dtm
import netCDF4 as nc
import numpy.ma as ma
import dateutil.parser as dparser
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.animation as animation


In [2]:
eps1 = np.load('/mnt/storage0/jacquielee/data/eps_ANHA4-EXH004_y2004m09d07.npy')
tracers = nc.Dataset('/mnt/storage0/jacquielee/research/data/Jasper/ANHA4-EXH004_y2004m09d07_gridT.nc')
sal = tracers.variables['vosaline'][0,:,:,:]
depth = tracers.variables['deptht']

eps0 = np.ma.masked_outside(eps1[:,:,:],0,100)

eps00 = np.ma.filled(eps0,fill_value = 10**(-16))
salinity = np.ma.masked_where(sal[:,:,:] == 0,sal)
salmask = (ma.getmaskarray(salinity[:,:]))*1
eps = ma.array(eps00,mask=salmask)

In [3]:
def make_figure(ax,cmap,depth,eps,grid):
    cmap.set_bad('burlywood')
    mesh = ax.pcolormesh(eps[grid,:,:],norm=colors.LogNorm(vmin=10**(-16), vmax=1),cmap=cmap)
    cbar = fig.colorbar(mesh,ax=ax)
    cbar.set_label('W/m^2')
    plt.axis((0, eps.shape[2], 0, eps.shape[1]))
    ax.grid()
    text = ax.set_title('Turbulent Dissipation Rate at {d:.2f} m'.format(d=depth[grid]))
    ax.set_xlabel('x Index')
    ax.set_ylabel('y Index')
    
    plot_ob = {'CONT_OBJ':mesh,'TEXT_OBJ':text}
    
    return plot_ob

In [4]:
def update_figure(grid_ind,depth,ax,cmap,eps,PLOT_OBJS):
#     for C in PLOT_OBJS['CONT_OBJ'].collections:
#         C.remove()
    del(PLOT_OBJS['CONT_OBJ'])
    mesh = ax.pcolormesh(eps[grid_ind,:,:],norm=colors.LogNorm(vmin=10**(-16), vmax=1),cmap=cmap)
    PLOT_OBJS['CONT_OBJ'] = mesh
    PLOT_OBJS['TEXT_OBJ'].set_text('Turbulent Dissipation Rate at {d:.2f} m'.format(d=depth[grid_ind]))
    
    return PLOT_OBJS

In [5]:
z = np.arange(0,50)
## create the time index
startgrid = z[0]
## create figure window
fig, ax = plt.subplots(1,1,figsize=(10,10))
## define colour map
cmap = plt.get_cmap('jet')

PLOT_OBJS = make_figure(ax,cmap,depth,eps,startgrid)

#print(PLOT_OBJS['CONT_OBJ'].collections)
def next_frame(t,PLOT_OBJS):
    #print(t)
    PLOT_OBJS = update_figure(t,depth,ax,cmap,eps,PLOT_OBJS)
    return PLOT_OBJS


#animate
ANI = animation.FuncAnimation(fig, next_frame, fargs=[PLOT_OBJS],frames=50,repeat=False)
plt.show()
mywriter = animation.FFMpegWriter(fps=12,bitrate=10000)
ANI.save('eps_animation1.mp4',writer=mywriter)